In [1]:
import argparse
import torch
import numpy as np
import os
from torch.nn import functional as F
import random
from tqdm import tqdm
from matplotlib import pyplot as plt
from torch.utils.data import Dataset, random_split
from torch.utils.data.dataloader import DataLoader
import pandas as pd
import torch.nn as nn
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pickle
import joblib
import math
from copy import *
c = deepcopy
device = torch.device('cuda:0')
import time
from torch import optim
from data.DataProcess import MyDataset   



from data.mydata_get import my_TrajectoryDataset_csv    
from model.my_E_D_STGCN import make_model
from model import scikit_wrappers

In [2]:
from torch.distributions import Normal

In [3]:
parser = argparse.ArgumentParser()


parser.add_argument('--obs_len', type=int, default=15) 
parser.add_argument('--pred_len', type=int, default=7) 
parser.add_argument('--teacher_len', type=int, default=3) 
parser.add_argument('--feature_num', type=int, default=8) 
parser.add_argument('--window_skip', type=int, default=1) 
parser.add_argument('--rel_if', type=bool, default=False) 
parser.add_argument('--data_delete_if', type=bool, default=True)  
parser.add_argument('--data_get_if', type=bool, default=False) 
parser.add_argument('--dec_in', type=int, default=8) 
parser.add_argument('--rep_triloss_len', type=int, default=20) 



parser.add_argument('--use_generator', type=bool, default=True) 
parser.add_argument('--use_LayerNorm', type=bool, default=True) 
parser.add_argument('--residual_connection', type=bool, default=True) 

parser.add_argument('--dropout', type=int, default=0.3) 
parser.add_argument('--num_layers_E', type=int, default=1)
parser.add_argument('--num_layers_d', type=int, default=1) 

parser.add_argument('--layernorm_size', type=int, default=8) 

parser.add_argument('--dim_hid', type=int, default=192) 
parser.add_argument('--dim_out', type=int, default=2) 
parser.add_argument('--dilation_factor', type=int, default=2) 



parser.add_argument('--MSE_if', type=bool, default=True) 
parser.add_argument('--representation_if', type=bool, default=True) 
parser.add_argument('--Encoder_ESG', type=bool, default=True) 
parser.add_argument('--Encoder_SGCN', type=bool, default=True) 
parser.add_argument('--output_informer', type=bool, default=False) 
parser.add_argument('--output_TCN', type=bool, default=False) 
parser.add_argument('--output_LSTM', type=bool, default=True) 



parser.add_argument('--decoder_embedding', type=bool, default=True) 
parser.add_argument('--encoder_embedding', type=bool, default=True) 

# parser.add_argument('--Flag', type=str, default='train') 
parser.add_argument('--train_without_stand', type=bool, default=False) 
parser.add_argument('--do_train', type=bool, default=False) 
parser.add_argument('--do_train_from_last', type=bool, default=False) 
parser.add_argument('--epochs', type=int, default=150) #  
parser.add_argument('--early_stop',type=bool,default=True,help='')
parser.add_argument('--early_stop_steps',type=int,default=3,help='')
parser.add_argument('--batch_size', type=int, default=1) 
parser.add_argument('--batch_loss', type=int, default=1,  
                    help='minibatch size')

parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=True) 
args, _ = parser.parse_known_args()

In [4]:
my_data_dir='./data/raw-30-day.csv'

In [5]:
if args.representation_if:
    args.feature_num=160
    args.dec_in=160
if args.output_LSTM:
    args.decoder_embedding=False

In [6]:
seed = 1693  


random.seed(seed)


np.random.seed(seed)


torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [7]:
df = pd.read_csv(my_data_dir)

In [10]:
grouped_first = df.groupby('track', sort=False)

In [11]:
def my_judge(C,B,A):
    print('A',A)
    print('B',B)
    print('C',C)
    BA = [A[0] - B[0], A[1] - B[1]]
    BC = [C[0] - B[0], C[1] - B[1]]
    print('BA',BA)
    print('BC',BC)
    dot_product = BA[0] * BC[0] + BA[1] * BC[1]
    dot_product = torch.round(dot_product * 10000) / 10000
    mod_BA = math.sqrt(BA[0]**2 + BA[1]**2)
    mod_BC = math.sqrt(BC[0]**2 + BC[1]**2)
    print('dot_product',dot_product)
    print('mod_AB * mod_BC',mod_BA * mod_BC)
    mod=mod_BA * mod_BC
    mod = round(mod, 4)
#     temp1=BA[0]+BC[0]+BA[1]+BC[1]
#     print('temp1',temp1)
#     if temp1==0:
#         angle_degree=180
#     else:
    angle = math.acos(dot_product / mod)
    angle_degree=math.degrees(angle)
    if angle_degree>90 :
        condition1=1
    else:
        condition1=0
    
    d_x=C[0]-B[0]
    d_y=C[1]-B[1]
    condition2=1
    if d_x>0.2 or d_y>0.2 :
        condition2=0
    if condition1 and condition2:
        return 1
    else:
        return 0

In [12]:
if args.data_get_if:
    if args.data_delete_if:
        df_new=[]
        for _, group in grouped_first:
        #     print(group.shape)
            df_train=torch.tensor(group.values)
            T_temp=df_train.shape[0]
            df_train_new=c(df_train)
            df_train_new[0,:]=df_train[0,:]
            df_train_new[1,:]=df_train[1,:]
            n=2
            for t_temp in range(2,T_temp):
                if my_judge(df_train[t_temp,2:4],df_train[t_temp-1,2:4],df_train[t_temp-2,2:4]):
                    df_train_new[n,:]=df_train[t_temp,:]
                    n=n+1

            df_train_new=df_train_new[:n,:]

            df_new.append(df_train_new)


        df_new = torch.cat(df_new, dim=0)    

        df=pd.DataFrame(df_new.numpy())

In [13]:
# print('df_new.shape',df_new.shape)   

In [14]:

scaler = StandardScaler()
columns_to_normalize = df.columns[2:]
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])
#         joblib.dump(scaler, 'scaler.joblib')
mean_Longitude = scaler.mean_[0]
std_Longitude = scaler.scale_[0]
mean_Latitude = scaler.mean_[1]
std_Latitude = scaler.scale_[1]

In [20]:
if args.data_delete_if and args.data_get_if:
    print(df.shape)
    grouped = df.groupby(0, sort=False)
    len(grouped)
else:
    print(df.shape)
    grouped = df.groupby('track', sort=False)
    len(grouped)

(284030, 12)


In [21]:
cuda=True
gpu=0

In [22]:
hyperparameters = {
    "batch_size": 64,            #三联损失时，一个batch的数量
    "channels": 30,
    "compared_length": None,
    "depth": 10,
    "nb_steps": 5,               # epoch次数
    "in_channels": 8,#---------
    "kernel_size": 3,
    "penalty": None,
    "early_stopping": None,
    "lr": 0.001,
    "nb_random_samples": 10,    # 负样本来源的batch数
    "negative_penalty": 1,
    "out_channels": 160,
    "reduced_size": 80,
    "cuda": cuda,
    "gpu": gpu
}

In [23]:
encoder_yearly = scikit_wrappers.CausalCNNEncoderClassifier()
encoder_yearly.set_params(**hyperparameters)

CausalCNNEncoderClassifier(batch_size=64, channels=30, compared_length=inf,
                           cuda=True, depth=10, in_channels=8, kernel_size=3,
                           nb_steps=5, out_channels=160, penalty=None,
                           reduced_size=80)

In [24]:
window=int(args.rep_triloss_len/2)

In [26]:
if args.data_get_if:
    if args.representation_if:
        args.feature_num=160
        args.dec_in=160
        data_loss = MyDataset(grouped, x_step=args.rep_triloss_len,y_step=7)  # x的时间段为21，y的时间段为7
        data_loss_x=data_loss[:][0]
        print(data_loss_x.shape)
        data_loss_x = data_loss_x.permute(0, 2, 1)
        data_loss_x = data_loss_x.numpy()
        data_loss_x = data_loss_x.astype(np.float64)
        encoder_yearly.fit_encoder(data_loss_x, save_memory=True, verbose=True)

In [27]:


if args.data_get_if:
    if args.representation_if:        
        represent_data=[]
        for _, group in grouped:
        #     print(group.shape)
            df_train=torch.tensor(group.values)
            df_train=df_train.unsqueeze(0)
            df_train = df_train.permute(0, 2, 1)
            df_train=df_train[:,:10,:]                  # 10表示 2个id和时间戳  + 8维特征
            print('df_train',df_train.shape)
            train_features_day = encoder_yearly.encode_window(df_train, window)
            print('train_features_day',train_features_day.shape)
            represent_data.append(train_features_day)

        represent_data = torch.cat(represent_data, dim=2)
        print(represent_data.shape)
        represent_data=represent_data.squeeze()
        represent_data = represent_data.permute(1, 0)
        print(represent_data.shape)
        represent_data = pd.DataFrame(represent_data.numpy())
        data_traj=represent_data
    else:
    #     data_traj=df
        no_represent_data=[]                 # 为了与表示学习的数据量一致，便于可视化对比，这边做一个窗口删减
        for _, group in grouped:
        #     print(group.shape)
            df_train=torch.tensor(group.values)
            df_train=df_train.unsqueeze(0)
            df_train = df_train.permute(0, 2, 1)
            df_train=df_train[:,:10,:]                  # 10表示 2个id和时间戳  + 8维特征
            print('df_train',df_train.shape)
    #         train_features_day=df_train[:,:,:np.shape(df_train)[2] - window+1]
            train_features_day=df_train[:,:,window-1:]
    #         train_features_day = encoder_yearly.encode_window(df_train, 20)
    #         print('train_features_day',train_features_day.shape)
            no_represent_data.append(train_features_day)
        no_represent_data = torch.cat(no_represent_data, dim=2)
        print(no_represent_data.shape)
        no_represent_data=no_represent_data.squeeze()
        no_represent_data = no_represent_data.permute(1, 0)
        print(no_represent_data.shape)
        no_represent_data = pd.DataFrame(no_represent_data.numpy())
        data_traj=no_represent_data     

In [28]:
    
        
if args.data_get_if:   
    dset = my_TrajectoryDataset_csv(
        data_traj,args,
        obs_len=args.obs_len,
        pred_len=args.pred_len,
        skip=args.window_skip,
        rel_if=args.rel_if)
    train_ratio = 0.6  # 训练集比例
    val_ratio = 0.2    # 验证集比例
    test_ratio = 0.2   # 测试集比例

    # 计算划分的样本数量
    total_samples = len(dset)
    train_size = int(train_ratio * total_samples)
    val_size = int(val_ratio * total_samples)
    test_size = total_samples - train_size - val_size


    torch.manual_seed(3407)
    train_dataset, val_dataset, test_dataset = random_split(dset, [train_size, val_size, test_size])
    if args.representation_if:
        with open('./data/rep/train_dataset.pkl', 'wb') as f:
            pickle.dump(train_dataset, f)
        with open('./data/rep/val_dataset.pkl', 'wb') as f:
            pickle.dump(val_dataset, f)
        with open('./data/rep/test_dataset.pkl', 'wb') as f:
            pickle.dump(test_dataset, f)
    else:
        with open('./data/train_dataset.pkl', 'wb') as f:
            pickle.dump(train_dataset, f)
        with open('./data/val_dataset.pkl', 'wb') as f:
            pickle.dump(val_dataset, f)
        with open('./data/test_dataset.pkl', 'wb') as f:
            pickle.dump(test_dataset, f)
        
        
else:
    if args.representation_if:
        with open('./data/rep/train_dataset.pkl', 'rb') as f:
            train_dataset= pickle.load(f)
        with open('./data/rep/val_dataset.pkl', 'rb') as f:
            val_dataset= pickle.load(f)
        with open('./data/rep/test_dataset.pkl', 'rb') as f:
            test_dataset= pickle.load(f)
    else:
        with open('./data/train_dataset.pkl', 'rb') as f:
            train_dataset= pickle.load(f)
        with open('./data/val_dataset.pkl', 'rb') as f:
            val_dataset= pickle.load(f)
        with open('./data/test_dataset.pkl', 'rb') as f:
            test_dataset= pickle.load(f)

In [29]:
for cnt, batch in enumerate(train_dataset):
    print(cnt)
    print(len(batch))
    print(batch[0].shape)  # T-N-F
    print(batch[1].shape)
    print(batch[0])
    print(batch[1])
#     print(batch[0])
    break

0
2
torch.Size([15, 2, 160])
torch.Size([7, 2, 160])
tensor([[[ 2.0260e-01,  1.9570e-01, -4.9400e-02,  ..., -4.7200e-02,
           7.0000e-04,  1.5610e-01],
         [-1.4950e-01,  8.7120e-01,  9.7500e-02,  ..., -2.8300e-02,
          -6.8500e-02,  2.0470e-01]],

        [[ 1.8790e-01,  2.0250e-01, -3.4100e-02,  ..., -3.8800e-02,
          -2.3000e-03,  1.6940e-01],
         [-1.5780e-01,  8.6550e-01,  6.6200e-02,  ..., -4.1700e-02,
          -2.9400e-02,  1.9470e-01]],

        [[ 1.7520e-01,  1.9380e-01, -2.4100e-02,  ..., -7.0400e-02,
          -1.4000e-02,  1.7460e-01],
         [-1.7280e-01,  8.6230e-01,  6.5100e-02,  ..., -6.5100e-02,
          -1.4100e-02,  2.0390e-01]],

        ...,

        [[-2.0000e-03,  1.4100e-01, -6.2000e-02,  ..., -1.9600e-02,
           1.3900e-02,  1.3170e-01],
         [-4.2490e-01,  7.6140e-01, -6.9900e-02,  ...,  8.6000e-03,
           1.2590e-01,  7.1900e-02]],

        [[-1.9800e-02,  1.3520e-01, -8.4400e-02,  ...,  1.2000e-03,
           2.9500

In [30]:

train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True) # 在不同epoch再次打乱
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)  # 这里不打乱，从而方便可视化复现

In [32]:
model=make_model(device,dilation_factor=args.dilation_factor,Encoder_SGCN=args.Encoder_SGCN,Encoder_ESG=args.Encoder_ESG,output_TCN=args.output_TCN,output_informer=args.output_informer,output_LSTM=args.output_LSTM,
                 encoder_embedding=args.encoder_embedding,decoder_embedding=args.decoder_embedding,
                 dec_in=args.dec_in,dim_hid=args.dim_hid,dim_out=args.dim_out,obs_len=args.obs_len,pred_len=args.pred_len,  
                 layernorm_size=args.layernorm_size, 
                 num_layers_E=args.num_layers_E,num_layers_d=args.num_layers_d,dropout=args.dropout,residual_connection=args.residual_connection,
                 use_LayerNorm=args.use_LayerNorm,use_generator=args.use_generator)

In [33]:
def masked_mae(preds, labels, null_val=np.nan):
    if np.isnan(null_val):
        mask = ~torch.isnan(labels)
    else:
        mask = (labels!=null_val)
    mask = mask.float()
    mask /=  torch.mean((mask))
    mask = torch.where(torch.isnan(mask), torch.zeros_like(mask), mask)
  
    loss = torch.abs(preds-labels)
    loss = loss * mask
    loss = torch.where(torch.isnan(loss), torch.zeros_like(loss), loss)
    return torch.mean(loss)

In [34]:
# criterion_train=my_mae().to(device)
criterion = nn.modules.L1Loss(reduction='mean').to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.0001)

In [35]:
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)

In [36]:
def my_mae(preds, labels, null_val=np.nan):   # B-T-N-2
    all_loss=0
    for t_pred in range(args.pred_len):
        loss = F.mse_loss(preds[:,t_pred,:,:],labels[:,t_pred,:,:], reduction='mean').sqrt()
        loss = loss*np.exp(-t_pred)
        all_loss=all_loss+loss
    all_loss=all_loss/(np.exp(0)+np.exp(-1)+np.exp(-2)+np.exp(-3)+np.exp(-4)+np.exp(-5)+np.exp(-6))
    loss_var=criterion(torch.var(preds, dim=1),torch.var(labels, dim=1))
    my_loss=all_loss+0.01*loss_var
    return my_loss

In [37]:
nParams = sum([p.nelement() for p in model.parameters()])
nParams

2476504

In [42]:
if args.output_TCN:
    save_folder = os.path.join('./saves/TCN')
elif args.output_LSTM:
    save_folder = os.path.join('./saves/LSTM')
elif  args.output_informer:
    save_folder = os.path.join('./saves/informer')
else :
    save_folder = os.path.join('./saves')
print(save_folder)
if args.representation_if:
    save_folder = os.path.join(save_folder,'representation')
if args.Encoder_ESG:
    save_folder = os.path.join(save_folder,'Encoder_ESG')
if args.Encoder_SGCN:
    save_folder = os.path.join(save_folder,'Encoder_SGCN')
if args.MSE_if:
    save_folder = os.path.join(save_folder,'MSE')
print(save_folder)


./saves/LSTM
./saves/LSTM\representation\Encoder_ESG\Encoder_SGCN\MSE


In [43]:
if not os.path.exists(save_folder):
    os.makedirs(save_folder)
model_path = os.path.join(save_folder,'best-model.pt')

In [44]:
print(model_path)

./saves/LSTM\representation\Encoder_ESG\Encoder_SGCN\MSE\best-model.pt


In [45]:
do_train=args.do_train ## 更换为True后，会覆盖当前保存模型

In [46]:
train_length=len(train_dataloader)
val_length=len(val_dataloader)
test_length=len(test_dataloader)

In [47]:
def val(data, model, criterion, optim, batch_size):
    print('begin valing')
    model.eval()
    total_loss =[]

    iter = 0
    for cnt, batch in enumerate(data):
        labels=batch[1][:,:,:,:2].to(device)
        encoder_raw_input=batch[0].to(device)
        if args.output_informer:
            dec_inp_zero = torch.zeros([batch[1].shape[0], args.pred_len,batch[1].shape[2],batch[1].shape[3]]).float()
            decoder_raw_input=torch.cat([batch[0][:,-args.teacher_len:,:,:], dec_inp_zero], dim=1).float().to(device)
            output=model(encoder_raw_input,decoder_raw_input)
            pred=output[:,-args.pred_len:,:,:]
        if args.output_LSTM: 
            decoder_raw_input=batch[0][:,-args.teacher_len:,:,:2].to(device)  # B-T_teacher-N-2
            ZC=model.my_channel_learn(encoder_raw_input)
            encoder_output = model.encode(ZC)                                # B-T-N-256
            Teacher=True
            decoder_output=model.decode(decoder_raw_input, encoder_output,labels,Teacher,'val')  # return:(pred_len, N,B, 2)
            pred=torch.permute(decoder_output,(2,0,1,3)).to(device)    # B-T-N-2
                     
        pred[:, :,:,0] = pred[:, :,:,0] * std_Longitude + mean_Longitude   ## 反向标准化
        labels[:, :,:,0]  = labels[:, :,:,0]  * std_Longitude + mean_Longitude
        pred[:, :,:,1]  = pred[:, :,:,1]  * std_Latitude + mean_Latitude
        labels[:, :,:,1] = labels[:, :,:,1]  * std_Latitude + mean_Latitude
        loss = F.mse_loss(pred, labels, reduction='mean').sqrt()+F.l1_loss(pred, labels)+F.mse_loss(pred, labels)+F.mse_loss(pred[:,-1], labels[:,-1], reduction='mean').sqrt()
        loss=loss/4
        total_loss.append(loss.item())


            
        iter += 1
    return np.mean(total_loss) 

In [48]:
def train(data, model, criterion, optim, batch_size,epoch_start_time):
    print('begin training')
    model.train()
    optim.zero_grad()
    train_total_loss =[]
    batch_count = 0
    is_fst_loss = True
    if args.use_amp:
        GradScaler = torch.cuda.amp.GradScaler()
        
    turn_point = int(train_length / args.batch_loss) * args.batch_loss + train_length % args.batch_loss - 1
    
    iter = 0
    for cnt, batch in enumerate(data):
        batch_count += 1
        model.zero_grad()
        labels=batch[1][:,:,:,:2].to(device)   # B-7-N-2
        encoder_raw_input=batch[0].to(device)
        if args.output_informer:
            dec_inp_zero = torch.zeros([batch[1].shape[0], args.pred_len,batch[1].shape[2],batch[1].shape[3]]).float()
            decoder_raw_input=torch.cat([batch[0][:,-args.teacher_len:,:,:], dec_inp_zero], dim=1).float().to(device)
            output=model(encoder_raw_input,decoder_raw_input)
            pred=output[:,-args.pred_len:,:,:]
        if args.output_LSTM: 
            decoder_raw_input=batch[0][:,-args.teacher_len:,:,:2].to(device)  # B-T_teacher-N-2
            ZC=model.my_channel_learn(encoder_raw_input)
            encoder_output = model.encode(ZC)                                # B-T-N-256
            Teacher=False
            decoder_output=model.decode(decoder_raw_input, encoder_output,labels,Teacher,'train')  # return:(pred_len, N,B, 2)
            pred=torch.permute(decoder_output,(2,0,1,3)).to(device)    # B-T-N-2
#             print(decoder_output)
#             print('decoder_raw_input.shape',decoder_raw_input.shape)
#             print('pred',pred)
#             print('labels',labels)
        if args.train_without_stand :            
            pred[:, :,:,0] = pred[:, :,:,0] * std_Longitude + mean_Longitude   ## 反向标准化
            labels[:, :,:,0]  = labels[:, :,:,0]  * std_Longitude + mean_Longitude
            pred[:, :,:,1]  = pred[:, :,:,1]  * std_Latitude + mean_Latitude
            labels[:, :,:,1] = labels[:, :,:,1]  * std_Latitude + mean_Latitude
        if args.MSE_if:
            loss = my_mae(pred,labels)                  
        else:
            loss = F.mse_loss(pred, labels, reduction='mean').sqrt()
            
        if args.use_amp:
            optimizer.zero_grad()
            GradScaler.scale(loss).backward()
            train_total_loss.append(loss.item())
            GradScaler.step(optim)
            GradScaler.update()
        else:
            loss.backward()
            train_total_loss.append(loss.item())
            grad_norm = optim.step()
            
        for name, param in model.named_parameters():
            if param.grad is not None:
                if torch.isinf(param.grad).any():
                    print(f"Parameter {name} has gradients containing infinite values.")

#         print()
#         if batch_count % args.batch_loss != 0 and cnt != turn_point:
#             l = criterion(pred,labels)   

#             if is_fst_loss:
#                 loss = l
# #                 print('here',loss)
#                 is_fst_loss = False
#             else:
#                 loss += l
        
#         else:
#             loss = loss / args.batch_loss
#             is_fst_loss = True
#             loss.backward()        
#             train_total_loss.append(loss.item())
#             grad_norm = optim.step()        
        
        t2 = time.time()
        if iter%100==0:

            print('train batch %s / %s, loss: %.2f' % (iter + 1, train_length, loss.item()))
            print('time: %.2f'%(t2-epoch_start_time))
        iter += 1
    return np.mean(train_total_loss) 


In [49]:
if args.do_train_from_last:
    with open(model_path, 'rb') as f:
        model = torch.load(f)

In [50]:
# with open(model_path, 'rb') as f:
#     model = torch.load(f)

In [51]:
if do_train:
    best_val = 10000000
    best_epoch = 10000
    for epoch in range(1, args.epochs + 1, 1):
        print('epoch:',epoch,flush=True)
        epoch_start_time = time.time()

        train_loss= train(train_dataloader, model, criterion, optimizer, args.batch_size,epoch_start_time)
        print('train_loss:',train_loss)

    # #————————————————————————————————————————————————————————

        val_loss= val(val_dataloader, model, criterion, optimizer, args.batch_size)
        print(
            '| end of epoch {:3d} | time: {:5.2f}s | train_loss {:5.4f} | valid rse {:5.4f} '.format(
                epoch, (time.time() - epoch_start_time), train_loss,val_loss), flush=True)

        scheduler.step(val_loss)
    # #————————————————————————————————————————————————————————
        if val_loss < best_val:
            print('save the model at epoch ' + str(epoch)+'*********')
            with open( model_path, 'wb') as f:
                print('save model epoch%s',epoch)
                torch.save(model, f)    
            best_val = val_loss
            best_epoch = epoch
#         elif args.early_stop and  epoch - best_epoch > args.early_stop_steps:
#             print('best epoch:', best_epoch)
#             raise ValueError('Early stopped.')
    # #————————————————————————————————————————————————————————

    print('best epoch:', best_epoch)   
else:
    with open(model_path, 'rb') as f:
        model = torch.load(f)

In [52]:
if args.output_TCN:
    result_folder = os.path.join('./results/TCN')
elif args.output_LSTM:
    result_folder = os.path.join('./results/LSTM')
elif args.output_informer:
    result_folder = os.path.join('./results/informer')    
else :
    result_folder = os.path.join('./results')
    
if args.representation_if:
    result_folder = os.path.join(result_folder,'representation')    
if args.Encoder_ESG:
    result_folder = os.path.join(result_folder,'Encoder_ESG')
if args.Encoder_SGCN:
    result_folder = os.path.join(result_folder,'Encoder_SGCN')
if args.MSE_if:
    result_folder = os.path.join(result_folder,'MSE')
if not os.path.exists(result_folder):
    os.makedirs(result_folder)
result_path_txt = os.path.join(result_folder,'results.txt')
result_path_figure= os.path.join(result_folder,'figure')

In [53]:
# with open(model_path, 'rb') as f:
#     model = torch.load(f)

In [103]:
def calculate_ade(predicted_traj, true_traj):

    displacement = torch.sqrt(torch.sum((predicted_traj - true_traj) ** 2, dim=2))
    
    ade = torch.mean(displacement)
    
    return ade

In [104]:
prediction = []
real=[]
history=[]
prediction_prob=[]

In [105]:
photo_gailv=False

In [106]:
model.eval()
total_loss =[]
total_MAE =[]
total_MSE =[]
total_ADE =[]
total_FDE =[]
iter = 0
for cnt, batch in enumerate(test_dataloader):
    labels=batch[1][:,:,:,:2].to(device)
    encoder_raw_input=batch[0].to(device)
    if args.output_informer:
        dec_inp_zero = torch.zeros([batch[1].shape[0], args.pred_len,batch[1].shape[2],batch[1].shape[3]]).float()
        decoder_raw_input=torch.cat([batch[0][:,-args.teacher_len:,:,:], dec_inp_zero], dim=1).float().to(device)
        output=model(encoder_raw_input,decoder_raw_input)
        pred=output[:,-args.pred_len:,:,:]
        pred[:, :,:,0] = pred[:, :,:,0] * std_Longitude + mean_Longitude   
        pred[:, :,:,1]  = pred[:, :,:,1]  * std_Latitude + mean_Latitude
    if args.output_LSTM: 
        decoder_raw_input=batch[0][:,-args.teacher_len:,:,:2].to(device)  # B-T_teacher-N-2
        ZC=model.my_channel_learn(encoder_raw_input)
        encoder_output = model.encode(ZC)                                # B-T-N-256
        Teacher=True
        if photo_gailv:
            decoder_output,log_prob=model.decode(decoder_raw_input, encoder_output,labels,Teacher,'test')  # return:(pred_len, N,B, 2)
            pred_all=torch.permute(decoder_output,(2,0,1,3)).to(device)    # B-T-N-2
            log_prob=torch.permute(log_prob,(2,0,1,3)).to(device)    # B-T-N-1

            pred_all[:, :,:,0] = pred_all[:, :,:,0] * std_Longitude + mean_Longitude   
            pred_all[:, :,:,1]  = pred_all[:, :,:,1]  * std_Latitude + mean_Latitude
            pred=pred_all[:1,:,:,:]
        else:
            decoder_output=model.decode(decoder_raw_input, encoder_output,labels,Teacher,'train')  # return:(pred_len, N,B, 2)
            pred=torch.permute(decoder_output,(2,0,1,3)).to(device)    # B-T-N-2
            pred[:, :,:,0] = pred[:, :,:,0] * std_Longitude + mean_Longitude   
            pred[:, :,:,1]  = pred[:, :,:,1]  * std_Latitude + mean_Latitude
            
    labels[:, :,:,0]  = labels[:, :,:,0]  * std_Longitude + mean_Longitude

    labels[:, :,:,1] = labels[:, :,:,1]  * std_Latitude + mean_Latitude
#     print(pred.shape)
#     print(labels.shape)

    loss = criterion(pred,labels)                  
    total_loss.append(loss.item())
    
    mae =F.l1_loss(pred, labels)
    total_MAE.append(mae.item())    
    
    mse = F.mse_loss(pred, labels)
    total_MSE.append(mse.item())
    
    ade=calculate_ade(pred, labels)
    total_ADE.append(ade.item())
    
    fde=calculate_ade(pred[:,-1], labels[:,-1])
    total_FDE.append(fde.item())
    
    if iter%100==0:
      
        print('test batch %s / %s, loss: %.2f' % (iter + 1, val_length, loss.item()))

    iter += 1
    

    if photo_gailv:
        prob_all = torch.cat((pred_all, log_prob), dim=-1)     # B-T-N-3
        prob_all=torch.permute(prob_all,(0,2,1,3)) # B-N-T-F    
        prediction_prob.append(prob_all.detach().cpu().numpy())   
    
    encoder_raw_input[:, :,:,0]  = encoder_raw_input[:, :,:,0]  * std_Longitude + mean_Longitude
    encoder_raw_input[:, :,:,1] = encoder_raw_input[:, :,:,1]  * std_Latitude + mean_Latitude
    
    
#     min_lon_x=min(encoder_raw_input[:, :,:,0])
#     min_lon_y=min(labels[:, :,:,0])
#     min_lon_p=torch.min(pred_all[:, :,:,0])
#     min_lon=min(min_lon_x,min_lon_y,min_lon_p)
    
    pred=torch.permute(pred,(0,2,1,3)) # B-N-T-F
    labels=torch.permute(labels,(0,2,1,3)) # B-N-T-F
    encoder_raw_input=torch.permute(encoder_raw_input,(0,2,1,3)) # B-N-T-F

    
    prediction.append(pred.detach().cpu().numpy())
    real.append(labels.detach().cpu().numpy())   
    history.append(encoder_raw_input[:,:,:,:2].detach().cpu().numpy())  

#     break 
    
print('test_loss:',np.mean(total_loss))
print('mae:',np.mean(total_MAE))
print('mse:',np.mean(total_MSE))
print('ade:',np.mean(total_ADE))
print('fde:',np.mean(total_FDE))
masked_mae=np.mean(total_loss)
with open(result_path_txt, 'a') as f:
    f.write(f'masked_mae:{masked_mae}')
    f.write('\n')
    f.close()

test batch 1 / 274, loss: 0.08
test batch 101 / 274, loss: 0.11
test batch 201 / 274, loss: 0.13
test_loss: 0.09271637993780049
mae: 0.09271637993780049
mse: 0.021686282812998715
ade: 0.165135026472536
fde: 0.2281297199834477
